In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client['recommender_system']
admissions = db['admissions']
patients = db['patients']
diagnoses_icd = db['diagnoses_icd']
procedures_icd = db['procedures_icd']
d_icd_diagnoses = db['d_icd_diagnoses']
d_icd_procedures = db['d_icd_procedures']
nies = db['nies']

In [3]:
df_admissions = pd.DataFrame(list(admissions.find()))
df_patients = pd.DataFrame(list(patients.find()))
df_diagnoses_icd = pd.DataFrame(list(diagnoses_icd.find()))
df_procedures_icd = pd.DataFrame(list(procedures_icd.find()))
df_d_icd_diagnoses = pd.DataFrame(list(d_icd_diagnoses.find()))
df_d_icd_procedures = pd.DataFrame(list(d_icd_procedures.find()))

In [4]:
df_nies = pd.DataFrame(list(nies.find()))
print('nies:', nies.count_documents({})) 
print(df_nies.head())

nies: 4776
                        _id  agegroup  Gender  sex  disability  ethnicgrp  \
0  6855811336e4026fa6fae289         3       1    1           0        0.0   
1  6855811336e4026fa6fae28a         2       2    2           0        0.0   
2  6855811336e4026fa6fae28b         1       2    2           0        0.0   
3  6855811336e4026fa6fae28c         2       1    1           1        0.0   
4  6855811336e4026fa6fae28d         3       1    1           1        0.0   

   HealthRegion  Source_of_Admission_Category  insure  medcard  ...  xadmtype  \
0             1                             1     1.0      0.0  ...         1   
1             1                             2     0.0      1.0  ...         1   
2             1                             1     1.0      1.0  ...         2   
3             1                             1     0.0      0.0  ...         1   
4             1                             1     0.0      1.0  ...         2   

   AdmTypeBinary  ScoreOVERALL  satisfa

In [6]:
print('admissions:', admissions.count_documents({}))
print(df_admissions.head())
print('patients:', patients.count_documents({}))
print(df_patients.head())
print('diagnoses_icd:', diagnoses_icd.count_documents({}))
print(df_diagnoses_icd.head())  
print('procedures_icd:', procedures_icd.count_documents({}))
print(df_procedures_icd.head())
print('d_icd_diagnoses:', d_icd_diagnoses.count_documents({}))
print(df_d_icd_diagnoses.head())
print('d_icd_procedures:', d_icd_procedures.count_documents({}))    
print(df_d_icd_procedures.head())

admissions: 283
                        _id  subject_id   hadm_id            admittime  \
0  67c4cb3a44a7421d696860bf    10046543  21402025  2155-03-15 21:22:00   
1  67c4cb3a44a7421d6968776b    10188106  28288574  2155-03-26 23:55:00   
2  67c4cb3a44a7421d6968776c    10188106  29174671  2155-03-22 17:26:00   
3  67c4cb3a44a7421d69687aa1    10207914  21364683  2155-03-22 05:06:00   
4  67c4cb3a44a7421d696880ba    10247468  21915776  2155-03-13 04:10:00   

             dischtime  admission_type                  admission_location  \
0  2155-03-20 17:00:00  EU OBSERVATION               WALK-IN/SELF REFERRAL   
1  2155-05-28 19:30:00          URGENT  INTERNAL TRANSFER TO OR FROM PSYCH   
2  2155-03-26 23:55:00        EW EMER.                      EMERGENCY ROOM   
3  2155-03-28 18:00:00        EW EMER.                      EMERGENCY ROOM   
4  2155-03-17 15:25:00        EW EMER.                      EMERGENCY ROOM   

  insurance language marital_status   race            edregtime  \
0  

Logic:
1. map addmission.hadm_id with patients to get demographics 
2. map addmission.hadm_id with diagnoses_icd.hadm_id get list of diagnoses
3. classify major category of diagnoses based on below list 

'Q45_QO1' - 'Tumour or cancer'
'Q45_QO2' - 'Heart condition'
 'Q45_QO3' - 'Lung condition'
 'Q45_QO4' - 'Neurological condition'
 'Q45_QO5' - 'Orthopaedic condition'
 'Q45_QO6' - 'COVID 19'
 'Q45_QO7' - 'Infection (other than COVID 19)'
 'Q45_QO8' - 'Digestive system condition'
 'Q45_QO9' - 'Diabetes and related problems'
 'Q45_QO10'- 'Adverse reaction or poising'
 'Q45_QO11' - 'Injury and or accident'
 'Q45_QO12'- 'Mental health issue'
 'Q45_QO13' - 'Tests and or investigations'
 'Q45_QO14' - 'Dont know or wasnt told'
 'Q45_QO15' - 'Other'

4. map satisfaction score for each patient by random sampling after applying filter with [admission_type, gender, insurance, age, icd_codes] 
5. get list of procedues for each patient
6. construct matrix
    1. rows: patient
    2. columns: procedures
    3. satisfaction index

In [7]:
merged_data = pd.merge(df_admissions, df_patients, on='subject_id', how='left')
diagnoses_merged = pd.merge(merged_data, df_diagnoses_icd, on=['hadm_id', 'subject_id'], how='left')

In [8]:
icd_to_category = {
    '250': 'Q45_QO9',  # Diabetes and related problems
    '390-459': 'Q45_QO2',  # Heart condition
    '460-519': 'Q45_QO3',  # Lung condition
    '320-389': 'Q45_QO4',  # Neurological condition
    '710-739': 'Q45_QO5',  # Orthopaedic condition
    'U07.1': 'Q45_QO6',  # COVID-19 (ICD-10 example, not in sample)
    '001-139': 'Q45_QO7',  # Infection (other than COVID-19)
    '520-579': 'Q45_QO8',  # Digestive system condition
    '140-239': 'Q45_QO1',  # Tumour or cancer
    '960-989': 'Q45_QO10',  # Adverse reaction or poisoning
    '800-999': 'Q45_QO11',  # Injury and or accident
    '290-319': 'Q45_QO12',  # Mental health issue
    'V01-V91': 'Q45_QO13',  # Tests and or investigations
    'Unknown': 'Q45_QO14'  # Dont know or wasnt told
}
category_labels = {
    'Q45_QO1': 'Tumour or cancer',
    'Q45_QO2': 'Heart condition',
    'Q45_QO3': 'Lung condition',
    'Q45_QO4': 'Neurological condition',
    'Q45_QO5': 'Orthopaedic condition',
    'Q45_QO6': 'COVID 19',
    'Q45_QO7': 'Infection (other than COVID 19)',
    'Q45_QO8': 'Digestive system condition',
    'Q45_QO9': 'Diabetes and related problems',
    'Q45_QO10': 'Adverse reaction or poising',
    'Q45_QO11': 'Injury and or accident',
    'Q45_QO12': 'Mental health issue',
    'Q45_QO13': 'Tests and or investigations',
    'Q45_QO14': 'Dont know or wasnt told',
    'Q45_QO15': 'Other'
}

In [9]:
def classify_diagnosis(icd_code):
    for code_range, category in icd_to_category.items():
        if '-' in code_range:
            start, end = code_range.split('-')
            try:
                if start <= str(icd_code) <= end:
                    return category_labels[category]
            except:
                continue
        elif code_range == str(icd_code):
            return category_labels[category]
    return category_labels['Q45_QO15']  # Default to 'Other'


In [10]:
diagnoses_merged['condition_label'] = diagnoses_merged['icd_code'].apply(classify_diagnosis)

In [11]:
diagnoses_grouped = diagnoses_merged.groupby(['hadm_id', 'subject_id'])['condition_label'].apply(lambda x: ', '.join(set(x))).reset_index()

In [12]:
patient_data = pd.merge(merged_data, diagnoses_grouped, on=['hadm_id', 'subject_id'], how='left')

In [13]:
df_nies['gender'] = df_nies['Gender'].map({1: 'M', 2: 'F'})
admission_type_map = {
    'EU OBSERVATION': 0.0,
    'URGENT': 0.0,
    'EW EMER.': 1.0
}
patient_data['AdmTypeBinary'] = patient_data['admission_type'].map(admission_type_map).fillna(0.0)

# Calculate average satisfaction_score by admission_type, gender, condition_label in df_nies
nies_avg_scores = df_nies.groupby(['AdmTypeBinary', 'gender', 'condition_label'])['satisfaction_score'].mean().reset_index()

In [14]:
patient_data = patient_data.assign(condition_label=patient_data['condition_label'].str.split(', ')).explode('condition_label')

In [15]:
# Merge with nies_avg_scores
patient_data = pd.merge(
    patient_data,
    nies_avg_scores,
    left_on=['AdmTypeBinary', 'gender', 'condition_label'],
    right_on=['AdmTypeBinary', 'gender', 'condition_label'],
    how='left'
)

In [16]:
# Aggregate satisfaction_score per hadm_id (average across conditions if multiple)
patient_data = patient_data.groupby(['hadm_id', 'subject_id', 'admission_type', 'insurance', 'gender', 'anchor_age', 'AdmTypeBinary'])['satisfaction_score'].mean().reset_index()

In [17]:
patient_data

,hadm_id,subject_id,admission_type,insurance,gender,anchor_age,AdmTypeBinary,satisfaction_score
0,20037816,12547294,EW EMER.,Medicare,F,22,1.0,0.853095
1,20067171,13584937,SURGICAL SAME DAY ADMISSION,Medicare,M,70,0.0,0.826236
2,20120850,15668092,EW EMER.,Other,M,71,1.0,0.894080
3,20165734,18879099,EU OBSERVATION,Medicaid,M,48,0.0,0.817131
4,20187769,11750274,EU OBSERVATION,Medicare,F,76,0.0,0.786521
...,...,...,...,...,...,...,...,...
278,29866935,15497616,EW EMER.,Medicare,M,71,1.0,0.868635
279,29880300,18845673,EW EMER.,Medicaid,M,42,1.0,0.858438
280,29880455,14677148,EU OBSERVATION,Other,F,59,0.0,0.768171
281,29953111,17079680,EW EMER.,Other,F,46,1.0,0.850469


In [18]:
procedures_merged = pd.merge(patient_data, df_procedures_icd, on=['hadm_id', 'subject_id'], how='left')

In [19]:
procedures_merged = procedures_merged.dropna(subset=['icd_code'])

In [20]:
procedures_merged.head()

,hadm_id,subject_id,admission_type,insurance,gender,anchor_age,AdmTypeBinary,satisfaction_score,_id,seq_num,chartdate,icd_code,icd_version
1,20067171,13584937,SURGICAL SAME DAY ADMISSION,Medicare,M,70,0.0,0.826236,684f6a3168f898b125637368,1.0,2155-03-17,3734,9.0
2,20067171,13584937,SURGICAL SAME DAY ADMISSION,Medicare,M,70,0.0,0.826236,684f6a3168f898b125637369,2.0,2155-03-17,3727,9.0
3,20120850,15668092,EW EMER.,Other,M,71,1.0,0.894080,684f69e968f898b125637183,1.0,2155-03-13,66,9.0
4,20120850,15668092,EW EMER.,Other,M,71,1.0,0.894080,684f69e968f898b125637184,2.0,2155-03-13,3607,9.0
5,20120850,15668092,EW EMER.,Other,M,71,1.0,0.894080,684f69e968f898b125637185,3.0,2155-03-13,45,9.0


In [21]:
procedures_grouped = procedures_merged.groupby(['hadm_id', 'subject_id'])['icd_code'].apply(lambda x: ', '.join(set(map(str, x)))).reset_index()

In [22]:
procedures_grouped

,hadm_id,subject_id,icd_code
0,20067171,13584937,"3734, 3727"
1,20120850,15668092,"41, 3607, 3722, 45, 8856, 66"
2,20276010,14969719,159
3,20296910,14629406,"741, 6529"
4,20329019,11069955,"3995, 5491, 9390, 3891, 3895, 4233, 3893, 4513"
...,...,...,...
158,29649567,11992390,5A1935Z
159,29686645,12701907,"0QSJ04Z, 0QSG04Z, 0QPJX5Z, 0QPGX5Z"
160,29866935,15497616,8051
161,29953111,17079680,4496


In [ ]:
# matrix construction
all_procedures = set()
for codes in procedures_merged['icd_code'].dropna():
    for code in str(codes).split(', '):
        all_procedures.add(code)

In [24]:
matrix = pd.pivot_table(
    procedures_merged,
    values='satisfaction_score',
    index=['hadm_id', 'subject_id', 'gender', 'anchor_age', 'admission_type', 'insurance'],
    columns='icd_code',
    aggfunc='mean',
    fill_value=0
)

In [25]:
# Save matrix to CSV
# matrix.to_csv('patient_procedure_matrix.csv', index=False)

# Display the matrix
print(matrix)

# Save processed patient data
# patient_data.to_csv('processed_patient_data.csv', index=False)

icd_code                                                                      14  \
hadm_id  subject_id gender anchor_age admission_type              insurance        
20067171 13584937   M      70         SURGICAL SAME DAY ADMISSION Medicare   0.0   
20120850 15668092   M      71         EW EMER.                    Other      0.0   
20276010 14969719   F      59         EW EMER.                    Other      0.0   
20296910 14629406   F      36         URGENT                      Medicaid   0.0   
20329019 11069955   F      70         EW EMER.                    Other      0.0   
...                                                                          ...   
29649567 11992390   F      29         EW EMER.                    Other      0.0   
29686645 12701907   M      20         SURGICAL SAME DAY ADMISSION Other      0.0   
29866935 15497616   M      71         EW EMER.                    Medicare   0.0   
29953111 17079680   F      46         EW EMER.                    Other     

In [26]:
matrix

,,,,,icd_code,14,17,34,40,41,45,46,59,66,159,...,8E0W4CZ,B2011ZZ,B2101ZZ,B2111ZZ,B211YZZ,B2131ZZ,B214YZZ,B2181ZZ,B218YZZ,B410YZZ
hadm_id,subject_id,gender,anchor_age,admission_type,insurance,,,,,,,,,,,,,,,,,,,,,
20067171,13584937,M,70,SURGICAL SAME DAY ADMISSION,Medicare,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20120850,15668092,M,71,EW EMER.,Other,0.0,0.0,0.0,0.0,0.89408,0.89408,0.0,0.0,0.89408,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20276010,14969719,F,59,EW EMER.,Other,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.85372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20296910,14629406,F,36,URGENT,Medicaid,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20329019,11069955,F,70,EW EMER.,Other,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29649567,11992390,F,29,EW EMER.,Other,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29686645,12701907,M,20,SURGICAL SAME DAY ADMISSION,Other,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29866935,15497616,M,71,EW EMER.,Medicare,0.0,0.0,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
